In [14]:
import pandas as pd
import numpy as np
import os
import io
import gzip


1. Lectura de los archivos en raw, y me creo los archivos parquet que voy a utilizar

In [ ]:
df_train = pd.read_csv('../../data/raw/train.csv')
df_test_nolabel = pd.read_csv('../../data/raw/test_nolabel.csv')
df_sample_submission= pd.read_csv('../../data/raw/sample_submission.csv')

# df_train = df_test_nolabel
#df_train = pd.read_parquet('../data/agregado_diario_2019-2021.parquet')

In [16]:
df_train.head()

,id,LoanNr_ChkDgt,Name,City,State,Bank,BankState,ApprovalDate,ApprovalFY,NoEmp,...,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementDate,DisbursementGross,BalanceGross,Accept
0,bd9d6267ec5,1523195006,"P-SCAPE LAND DESIGN, LLC",NORTHFIELD,OH,CITIZENS BANK NATL ASSOC,RI,1-Nov-05,2006,2,...,0,2,0,1,N,N,31-Dec-05,"$8,000.00",$0.00,1
1,9eebf6d8098,1326365010,The Fresh & Healthy Catering C,CANTON,OH,"FIRSTMERIT BANK, N.A.",OH,6-Jun-05,2005,2,...,1,2,1,1,N,N,31-Jul-05,"$166,000.00",$0.00,1
2,83806858500,6179584001,AARON MASON & HOWE LLC,SAWYERWOOD,OH,"PNC BANK, NATIONAL ASSOCIATION",OH,18-Mar-03,2003,2,...,4,2,1,2,Y,N,31-Mar-03,"$25,000.00",$0.00,1
3,a21ab9cb3af,8463493009,MID OHIO CAR WASH,COLUMBUS,OH,THE HUNTINGTON NATIONAL BANK,OH,28-Jun-95,1995,2,...,0,0,1,0,N,N,31-Jan-96,"$220,100.00",$0.00,1
4,883b5e5385e,3382225007,Bake N Brew LLC,Newark,OH,THE HUNTINGTON NATIONAL BANK,OH,16-Apr-09,2009,0,...,0,0,0,1,N,N,31-May-09,"$25,000.00",$0.00,0


In [21]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3284 entries, 0 to 3283
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 3284 non-null   object 
 1   LoanNr_ChkDgt      3284 non-null   int64  
 2   Name               3284 non-null   object 
 3   City               3284 non-null   object 
 4   State              3284 non-null   object 
 5   Bank               3279 non-null   object 
 6   BankState          3279 non-null   object 
 7   ApprovalDate       3284 non-null   object 
 8   ApprovalFY         3284 non-null   object 
 9   NoEmp              3284 non-null   int64  
 10  NewExist           3284 non-null   float64
 11  CreateJob          3284 non-null   int64  
 12  RetainedJob        3284 non-null   int64  
 13  FranchiseCode      3284 non-null   int64  
 14  UrbanRural         3284 non-null   int64  
 15  RevLineCr          3280 non-null   object 
 16  LowDoc             3275 

In [22]:
df_train.isnull().sum()

id                   0
LoanNr_ChkDgt        0
Name                 0
City                 0
State                0
Bank                 5
BankState            5
ApprovalDate         0
ApprovalFY           0
NoEmp                0
NewExist             0
CreateJob            0
RetainedJob          0
FranchiseCode        0
UrbanRural           0
RevLineCr            4
LowDoc               9
DisbursementDate     7
DisbursementGross    0
BalanceGross         0
dtype: int64

2. Cambiar los valores correspondientes.

La gran mayoría de los valores son de tipo objet, int64 y float64, tienen que ser acorde a sus valores.

Primero cambio los valores de las fechas, que son Approval Date y DisbursementDate que están puesto con letra escrita en vez de en el formato de numero.
Después cambiamos los valores que están raros y no se adaptan

* *DisbursementDate* => transformar la fecha
* *Name* => String, Si no sabemos lo ponemos Unknown
* *City* => String, Si no sabemos lo ponemos Unknown
* *State* => Son todosde tipo OH
* *Bank* => Si quitamos los None es de tipo String, Si no sabemos lo ponemos Unknown
* *BankState* => Si quitamos los None es de tipo String, Si no sabemos lo ponemos Unknown
* *NewExist* => 2 / 1 / 0, int, y poner la media donde sea nulo; 

    1 = Existing business, 2 = New business; 

    Si no lo sabemos ponemos vieja empresa que es la mayoría, es decir, que ya existía la empresa, 0. 

    Por lo que corresponde con 0 = Existing y 1 = New
* *RevLineCr* => array(['N', 'Y', '0', 'T', None, '`'], dtype=object, 

    Hay que limpiarlo, sabiendo que tiene que ser Y = Yes, N = No, para el que no sepamos ponemos la mayoría; después ponemos True => 1 y False => 0
* *LowDoc* => array(['N', 'Y', None, 'A', '0', 'S', 'R', 'C'], dtype=object)

    Tiene que ser Y = Yes, N = No, para el que no sepamos ponemos la mayoría; después ponemos True => 1 y False => 0
* *DisbursementGross* => Quitar el dolar y hacerlo double
* *BalanceGross* => Quitar el dolar y hacerlo double y donde sea nulo

In [23]:

df_train2 = df_train.copy()
df_train2['ApprovalDate'] = pd.to_datetime(df_train2['ApprovalDate'])
df_train2['DisbursementDate'] = pd.to_datetime(df_train2['DisbursementDate'])

df_train2['Name'].fillna('Unknown', inplace=True) # Le pongo desconocido
df_train2['City'].fillna(df_train2['City'].mode()[0], inplace=True) # La media
df_train2['Bank'].fillna('Unknown', inplace=True) # Desconocido / Mayoritario habría que elegir
df_train2['BankState'].fillna('Unknown', inplace=True) # Desconocido / Mayoritario habría que elegir
#Lo convertimos en Int y despues los Nan en numero, Nueva => True, 1; Si ya existe la ponemos como 0
df_train2['NewExist'] = df_train2['NewExist'].apply(
    lambda x: 0 if str(x).strip().replace('.0','') == '1'
              else (1 if str(x).strip().replace('.0','') == '2'
                    else np.nan))

df_train2['NewExist'] = df_train2['NewExist'].astype("Int64")
######################Limpieza RevLineCr#
df_train2['RevLineCr'] = df_train2['RevLineCr'].apply(lambda x: 1 if str(x).strip().upper() in ['Y','T'] 
                                                        else (0 if str(x).strip().upper() in ['N','0'] 
                                                              else np.nan))
df_train2['RevLineCr'] = df_train2['RevLineCr'].astype("Int64")
######################Limpieza LowDoc#
df_train2['LowDoc'] = df_train2['LowDoc'].apply(lambda x: 1 if str(x).strip().upper() in ['Y'] 
                                                  else (0 if str(x).strip().upper() in ['N','0'] 
                                                        else np.nan))
df_train2['LowDoc'] = df_train2['LowDoc'].astype("Int64")
######################
#ordenar por fecha
df_train2['DisbursementDate'].fillna(np.nan, inplace=True) 

#Limpiamos los dólares y lo ponemos en tipo float
df_train2['DisbursementGross'] = df_train2['DisbursementGross'].replace('[\$,]', '', regex=True).astype(float)
df_train2['BalanceGross'] = df_train2['BalanceGross'].replace('[\$,]', '', regex=True).astype(float)

########### Limpiar la columna 'ApprovalFY' para tener solo los años sin ninguna letra
df_train2['ApprovalFY'] = df_train2['ApprovalFY'].astype(str).str.extract('(\d+)', expand=False)

# Convertir a numérico (opcional, pero útil para operaciones posteriores)
df_train2['ApprovalFY'] = pd.to_numeric(df_train2['ApprovalFY'], errors='coerce')

# df_train2.to_csv('../../data/processed/df_train.csv')
df_train2.to_csv('../../data/processed/df_test_nolabel.csv')


<>:31: SyntaxWarning: invalid escape sequence '\$'
<>:32: SyntaxWarning: invalid escape sequence '\$'
<>:35: SyntaxWarning: invalid escape sequence '\d'
<>:31: SyntaxWarning: invalid escape sequence '\$'
<>:32: SyntaxWarning: invalid escape sequence '\$'
<>:35: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_74235/2713687183.py:31: SyntaxWarning: invalid escape sequence '\$'
  df_train2['DisbursementGross'] = df_train2['DisbursementGross'].replace('[\$,]', '', regex=True).astype(float)
/tmp/ipykernel_74235/2713687183.py:32: SyntaxWarning: invalid escape sequence '\$'
  df_train2['BalanceGross'] = df_train2['BalanceGross'].replace('[\$,]', '', regex=True).astype(float)
/tmp/ipykernel_74235/2713687183.py:35: SyntaxWarning: invalid escape sequence '\d'
  df_train2['ApprovalFY'] = df_train2['ApprovalFY'].astype(str).str.extract('(\d+)', expand=False)
/tmp/ipykernel_74235/2713687183.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling ba

In [ ]:
df_train2.isna().sum()

id                   0
LoanNr_ChkDgt        0
Name                 0
City                 0
State                0
Bank                 0
BankState            0
ApprovalDate         0
ApprovalFY           0
NoEmp                0
NewExist             0
CreateJob            0
RetainedJob          0
FranchiseCode        0
UrbanRural           0
RevLineCr            0
LowDoc               0
DisbursementDate     0
DisbursementGross    0
BalanceGross         0
Accept               0
dtype: int64

In [ ]:
df_train2.BalanceGross.value_counts(dropna=False)

BalanceGross
0.0         22833
996262.0        1
9111.0          1
Name: count, dtype: int64